In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")


# Any results you write to the current directory are saved as output.

# 前処理
## 欠損値の処理

In [ ]:
#年齢の補完(nameTitleを使ってやるとなお良いと思う)
#Titleから新しい特徴量を追加

# mean_age = int(df_train["Age"].mean())
# print(mean_age)

df_train["Age"] = df_train["Age"].fillna(29)
df_test["Age"] = df_test["Age"].fillna(29)


# df_test[df_test["Age"].isnull()]
    
# df["Age"] = np.where(df_test["Age"].isnull,
#                     np.where("Mr" in df_test["Name"],
#                             32))
    
        
        
        
# df_test[df_test["Age"].isnull()]      

In [ ]:
#乗港した港の補完
df_train[df_train["Embarked"].isnull()]
# 同じticket番号の人他にいないか確認
df_test[df_test["Embarked"]=="113572"]
#一番多いSで埋める
df_train.loc[df_train["PassengerId"].isin([62,830]),"Embarked"]="S"
df_train.loc[df_train["PassengerId"].isin([62,830])]

In [ ]:
# 運賃の補完
df_train[["Pclass","Fare"]].groupby("Pclass").mean()

df_test[df_test["Fare"].isnull()]

df_test.loc[df_test["PassengerId"]==1044,"Fare"]=13.675550
df_test.loc[df_test["PassengerId"]==1044]

In [ ]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())

## カテゴリ変数への変換

In [ ]:
#性別と乗港の変換（ダミー変数化）
df_train = pd.get_dummies(df_train, columns=["Sex"])
df_test = pd.get_dummies(df_test, columns=["Sex"])

df_train = pd.get_dummies(df_train, columns=["Embarked"])
df_test = pd.get_dummies(df_test, columns=["Embarked"])

df_train.head()

## 不要な特徴量の削除

In [ ]:
df_train.drop(["Name","Cabin","Ticket"], axis=1, inplace=True)
df_test.drop(["Name","Cabin","Ticket"], axis=1, inplace=True)

# ランダムフォレストで学習

In [ ]:
X_train = df_train.drop(["PassengerId", "Survived"], axis=1)
Y_train = df_train["Survived"]
X_test = df_test.drop("PassengerId", axis=1).copy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=1)

forest.fit(X_train, Y_train)

forest.score(X_train, Y_train)

In [ ]:
# k分割交差検証を用いて学習
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=3)

for train_index, test_index in kf.split(X_train, Y_train):
    X_cv_train = X_train.iloc[train_index]
    X_cv_test = X_train.iloc[test_index]
    Y_cv_train = Y_train.iloc[train_index]
    Y_cv_test = Y_train.iloc[test_index]
    forest = RandomForestClassifier(random_state=1)
    forest.fit(X_cv_train, Y_cv_train)
    predictions = forest.predict(X_cv_test)
    print(round(accuracy_score(Y_cv_test, forest.predict(X_cv_test))*100, 2))


In [ ]:
#提出
forest = RandomForestClassifier(random_state=1)
forest.fit(X_train, Y_train)
Y_prediction = forest.predict(X_test)
submission = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": Y_prediction
    })
submission.to_csv("submission.csv", index=False)